# Spine

In [16]:
import glob
import os
import shutil
import pydicom
import cv2
import json
import pandas as pd
import SimpleITK as sitk
import numpy as np
import sys
import math
from PIL import Image
from shapely.geometry import Polygon
from shapely.geometry import LinearRing
from skimage import draw
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import Input,Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, concatenate, Activation,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, SeparableConv2D, Add, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, History, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score


os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# Data_load

In [18]:
def train_data_loading(path,image_size = 512):
#     mkfolder(path)
    trainlist = glob.glob(path+'/*.dcm')

# 이미 한번 돌려서 저장해논거 다시 저장x 하기 위해 주석처리        
#     data_pre(trainlist)

    img_mask_path = '/home/hackerton/jupyter/Spine_data/A_img_mask/'
    
    train_img_path = img_mask_path +'/*.jpg'
    train_mask_path = img_mask_path +'/*.png'

# 이미 한번 돌려서 아래 주소로 대체
#     aug_path = augmentation(train_img_path,train_mask_path)
    
    aug_path = '/home/hackerton/jupyter/Spine_data/A_aug/'

    imgs_train, imgs_mask_train, imgs_name = create_train_data(aug_path, image_size, image_size, 'train', 'jpg')
    return imgs_train, imgs_mask_train, imgs_name

def data_pre(dcmlist):
# 이미 한번 돌려서 저장해논거 다시 저장x 하기 위해 주석처리    
    img_mask_path = '/home/hackerton/jupyter/Spine_data/A_img_mask/'
    mkfolder(img_mask_path)
    
    for i in range(len(dcmlist)):
        # try:
        if i != 133:   
            file_name = dcmlist[i].split('/')[-1].split('.')[0]
            dicompath = dcmlist[i]
            dicom = pydicom.read_file(dicompath)
            img = dicom.pixel_array
            img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
            # except:
            #     print(dicompath)
            #     continue
            pad_img = zero_padding(img)
            image = Image.fromarray(pad_img)
            image = image.convert('L')
            file_save = file_name + '.jpg'
            print(file_save)

            image.save(img_mask_path + file_save)
    print('Complete !')
    
    for dcm in dcmlist:
        jsonfile = dcm[:-4]+'.json'
        
        file_name = dcm.split('/')[-1].split('.')[0]
        
        reader = sitk.ReadImage(dcm)
        image_array = sitk.GetArrayFromImage(reader)
        height = reader.GetMetaData('0028|0010')
        width = reader.GetMetaData('0028|0011')
        data = []
        for line in open(jsonfile,'r'):
            if dcm !=  dcmlist[133]:
                data.append(json.loads(line))
        for json_data in data:
            mask = np.zeros((int(height), int(width)))
            if json_data['annotation']['ANNOTATION_DATA'] is not None:
                for m in json_data['annotation']['ANNOTATION_DATA']:
                    if 'm_points' in m:
                        a = []
                        for i in m['m_points']:
                            b = (i['x'], i['y'])
                            a.append(b)
                        r = LinearRing(a)
                        s = Polygon(r)
                        x, y = s.exterior.coords.xy
                        maskd = poly2mask(y, x, (int(height), int(width)))
                        mask = mask + maskd
                mask = mask*255
                mask = zero_padding(mask)
                mask = np.expand_dims(mask, axis=0)
                img = sitk.GetImageFromArray(mask.astype('uint8'))
                num = 0
                maskpath = img_mask_path + file_name +'.png'
                print(maskpath)
                sitk.WriteImage(img, maskpath)
            else:
                print('haha')


In [19]:
def augmentation(img_path,mask_path):
    img_li = sorted(glob.glob(img_path))
    mask_li = sorted(glob.glob(mask_path))
    print(len(img_li), len(mask_li))
    i=0

    for img, mask in zip(img_li, mask_li):
        if i%100==0:
            print('{}/{}'.format(i, len(img_li)))
        savepath = '/home/hackerton/jupyter/Spine_data/A_aug/'
        mkfolder(savepath)
        ori_img = cv2.imread(img, 0)
        mask_img = cv2.imread(mask, 0)
        img_name = img[img.rindex('/')+1:-4]
        mask_name = img[mask.rindex('/')+1:-4]
        print(img_name)
        print(mask_name)
        cv2.imwrite(savepath+'/{}.jpg'.format(img_name), cv2.resize(ori_img, (512,512)))
        cv2.imwrite(savepath+'/{}.png'.format(img_name), cv2.resize(mask_img, (512,512)))
        for j in range(9):
            aug_img, aug_mask = Augment_crop(ori_img, mask_img)        
            aug_img = cv2.resize(aug_img, (512,512))
            aug_mask = cv2.resize(aug_mask, (512,512))
            cv2.imwrite(savepath+'/{}_{}.jpg'.format(img_name, j), aug_img)
            cv2.imwrite(savepath+'/{}_{}.png'.format(img_name, j), aug_mask)
        i+=1
    return savepath

In [20]:
def create_train_data(train_path, out_rows, out_cols, name, img_type):
    print('-'*30)
    print('Creating training images...')
    print('-'*30)
    
    imgs = glob.glob(train_path +"*." + img_type)
    imgdatas = np.ndarray((len(imgs),out_rows,out_cols,1), dtype=np.uint8)
    imglabels = np.ndarray((len(imgs),out_rows,out_cols,1), dtype=np.uint8)
    imgnames=[]
    for i, imgname in enumerate(imgs):
        if i%1000==0:
            print('{}/{}'.format(i, len(imgs)))
        midname = imgname[imgname.rindex("/")+1:-4]     
        img = load_img(imgname, color_mode = "grayscale")
        label = load_img(imgname.replace('jpg', 'png'), color_mode = "grayscale")
        img=img.resize((out_rows,out_cols))
        label=label.resize((out_rows,out_cols))

        img = img_to_array(img)
        label = img_to_array(label)
        imgdatas[i] = img
        imglabels[i] = label
        imgnames.append(midname)
        
    imgdatas = imgdatas.astype('float32')
    imglabels = imglabels.astype('float32')
    
    print('img : ', imgdatas.max())
    print('mask : ',imglabels.max())
    
    print('-'*30)
    print('normalization start...')
    print('-'*30)
    imgdatas = imgdatas/255.0
    
    imglabels[imglabels <= 127] = 0
    imglabels[imglabels > 127] = 1
    
    
    print('img : ',imgdatas.max())
    print('mask : ',imglabels.max())
    print('mask : ',imglabels.min())
    
    print('loading done')
    return(imgdatas,imglabels,imgnames)

In [21]:
def zero_padding(img):
    y, x = img.shape
    if x > y:
        new_size = x
        add_size = x-y
        
        add_image = np.zeros((new_size, add_size))
        if add_size % 2==0:
            add_img = np.zeros((int(add_size/2), new_size))
            img = np.concatenate((add_img,img, add_img),axis=0)

        elif add_size % 2==1:
            add_img_top = np.zeros((int(add_size/2),new_size))
            add_img_bot = np.zeros((int(add_size/2)+1, new_size))
            img = np.concatenate((add_img_top,img, add_img_bot),axis=0)

    elif y > x:
        new_size = y
        add_size = y-x
        
        add_image = np.zeros((add_size, new_size))
        if add_size % 2==0:
            add_img = np.zeros((new_size, int(add_size/2)))
            img = np.concatenate((add_img,img, add_img),axis=1)

        elif add_size % 2==1:
            add_img_left = np.zeros((new_size, int(add_size/2)))
            add_img_right = np.zeros((new_size, int(add_size/2)+1))
            img = np.concatenate((add_img_left,img, add_img_right),axis=1)
    
    return img

def poly2mask(vertex_row_coords, vertex_col_coords, shape):
    fill_row_coords, fill_col_coords = draw.polygon(
        vertex_row_coords, vertex_col_coords, shape)
    mask = np.zeros(shape, dtype=np.bool)
    mask[fill_row_coords, fill_col_coords] = True
    return mask

def mkfolder(folder):
    if not os.path.lexists(folder):
        os.makedirs(folder)
    
def weight_center(img):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    cnt = contours[0] 
    mmt = cv2.moments(cnt) 
    cx = int(mmt['m10']/mmt['m00']) 
    cy = int(mmt['m01']/mmt['m00']) 
    print( 'x 무게중심', cx, 'y 무게중심', cy )
    return cx, cy
def randomRoate(img, label, p, angle_range):
    if len(img.shape)==3:
        height, width, channel = img.shape
    else:
        height, width = img.shape
    
    random_angle = np.random.randint(-angle_range/2, angle_range/2)*2
    matrix = cv2.getRotationMatrix2D((width/2, height/2), random_angle, 1)
    rotate_img = cv2.warpAffine(img, matrix, (width, height))
    rotate_label = cv2.warpAffine(label, matrix, (width, height))
    
    return rotate_img, rotate_label

def find_top_point(ori_img, mask_img):
    mask_img_T = mask_img.T

    tmp_index=[]
    for i, c in enumerate(mask_img_T):
        if len(np.unique(c))>1:
            tmp_index.append(i)
    x_min = min(tmp_index)
    x_max = max(tmp_index)

    p_x = int((x_min+x_max)/2) 
    p_y = np.where(mask_img_T[int((x_min+x_max)/2)]==255)[0][0]
    p = (p_x, p_y)

    return p_x, p_y

def Augment_crop(img, mask):
    p_x, p_y=find_top_point(img, mask)
    rotate_img, rotate_mask = randomRoate(img, mask, (p_x, p_y), 20)
    random_size = np.random.randint(15,35)*20 # 300-600
    
    h, w= img.shape
    x1 = p_x-random_size if p_x-random_size>0 else 0
    x2 = p_x+random_size if p_x+random_size<w else w
    y1 = p_y-random_size if p_y-random_size>0 else 0
    y2 = p_y+random_size if p_y+random_size<h else h
    
    crop_img = rotate_img[y1:y2,x1:x2]
    crop_mask = rotate_mask[y1:y2,x1:x2]

    return crop_img, crop_mask

def dice_coef(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

def dice_coef_loss(y_true, y_pred):
        return 1-dice_coef(y_true, y_pred)

def sens(y_true, y_pred): # sensitivity, recall
    print(y_pred)
    print(y_true)
    y_target_yn = K.round(K.clip(y_true, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall

def sch(epoch):
    if epoch>100 and epoch<=250:
        return 0.0001
    elif epoch>250:
        return 0.00001
    else:
        return 0.001

# Model Shape

In [22]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K

def hourglass_network(input_shape, num_classes, num_stacks, num_channels):
    
    input = Input(input_shape)

    start = front_module(input, num_channels)

    head_next_stage = start

    outs = []
    for i in range(num_stacks):
        head_next_stage, head_to_loss = hourglass_module(head_next_stage, num_classes, num_channels, i)
        outs.append(head_to_loss)
    
    x = BatchNormalization()(outs[-1])
    x = Conv2DTranspose(int(num_classes/2), (2, 2), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(int(num_classes/4), (2, 2), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(1, (1, 1), activation='sigmoid')(x)

    model = Model(inputs=input, outputs=x)

    return model

def hourglass_module(bottom, num_classes, num_channels, hgid):

    left_features = left_half_blocks(bottom, hgid, num_channels)

    rf1 = right_half_blocks(left_features, hgid, num_channels)
    
    head_next_stage, head_parts = heads(bottom, rf1, num_classes, hgid, num_channels)

    return head_next_stage, head_parts

def front_module(input, num_channels):
    
    x = Conv2D(64,kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu', name='front_conv_1x1_x1')(
        input)
    x = BatchNormalization()(x)
    x = bottleneck_block(x, int(num_channels // 2), 'front_residual_x1')
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = bottleneck_block(x, int(num_channels // 2), 'front_residual_x2')
    x = bottleneck_block(x, num_channels, 'front_residual_x3')

    return x

def heads(prelayerfeatures, rf1, num_classes, hgid, num_channels):
    
    head = Conv2D(num_channels, kernel_size=(1, 1), activation='relu', padding='same', name=str(hgid) + 'conv_1x1_x1')(
        rf1)
    head = BatchNormalization()(head)

    head_parts = Conv2D(num_classes, kernel_size=(1, 1), activation='linear', padding='same',
                        name=str(hgid) + 'conv_1x1_parts')(head)

    head = Conv2D(num_channels, kernel_size=(1, 1), activation='linear', padding='same',
                  name=str(hgid) + 'conv_1x1_x2')(head)
    head_m = Conv2D(num_channels, kernel_size=(1, 1), activation='linear', padding='same',
                    name=str(hgid) + 'conv_1x1_x3')(head_parts)

    head_next_stage = Add()([head, head_m, prelayerfeatures])
    return head_next_stage, head_parts


def bottleneck_block(bottom, num_out_channels, block_name):

    if K.int_shape(bottom)[-1] == num_out_channels:
        skip = bottom
    else:
        skip = Conv2D(num_out_channels, kernel_size=(1, 1), activation='relu', padding='same',
                       name=block_name + 'skip')(bottom)

    x = Conv2D(int(num_out_channels / 2), kernel_size=(1, 1), activation='relu', padding='same',
                name=block_name + 'conv_1x1_x1')(bottom)
    x = BatchNormalization()(x)
    x = Conv2D(int(num_out_channels / 2), kernel_size=(3, 3), activation='relu', padding='same',
                name=block_name + 'conv_3x3_x2')(x)
    x = BatchNormalization()(x)
    x = Conv2D(num_out_channels, kernel_size=(1, 1), activation='relu', padding='same',
                name=block_name + 'conv_1x1_x3')(x)
    x = BatchNormalization()(x)
    x = Add(name=block_name + 'residual')([skip, x])

    return x

def left_half_blocks(bottom, hglayer, num_channels):

    hgname = 'hg' + str(hglayer)

    f1 = bottleneck_block(bottom, num_channels, hgname + 'l1')
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(f1)

    f2 = bottleneck_block(x, num_channels, hgname + 'l2')
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(f2)

    f4 = bottleneck_block(x, num_channels, hgname + 'l4')
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(f4)

    f8 = bottleneck_block(x, num_channels, hgname + 'l8')

    return (f1, f2, f4, f8)

def left_to_right(left, right, name, num_channels):
    
    xleft = bottleneck_block(left, num_channels, name + 'connect')
    xright = UpSampling2D()(right)
    add = Add()([xleft, xright])
    out = bottleneck_block(add, num_channels, name + 'connect_conv')
    return out

def bottom_layer(lf8, hgid, num_channels):
    
    lf8_connect = bottleneck_block(lf8, num_channels, str(hgid) + "lf8")

    x = bottleneck_block(lf8, num_channels, str(hgid) + "lf8_x1")
    x = bottleneck_block(x, num_channels, str(hgid) + "lf8_x2")
    x = bottleneck_block(x, num_channels, str(hgid) + "lf8_x3")

    rf8 = Add()([x, lf8_connect])

    return rf8

def right_half_blocks(leftfeatures, hglayer, num_channels):  #branch_add
    lf1, lf2, lf4, lf8 = leftfeatures

    rf8 = bottom_layer(lf8, hglayer, num_channels)

    rf4 = left_to_right(lf4, rf8, 'hg' + str(hglayer) + 'rf4', num_channels)

    rf2 = left_to_right(lf2, rf4, 'hg' + str(hglayer) + 'rf2', num_channels)

    rf1 = left_to_right(lf1, rf2, 'hg' + str(hglayer) + 'rf1', num_channels)

    return rf1

In [29]:
class CosineAnnealingWarmup(tf.keras.callbacks.Callback):
    def __init__(self, epochs_per_cycle,iteration, max_lr, min_lr, verbose = 1):
        self.epochs_per_cycle = epochs_per_cycle
        self.max_lr = max_lr
        self.min_lr = min_lr
        self.iteration = iteration;
        self.steps = 0;
        self.learning_rate = max_lr;
        self.epochs = 0; # epoch to search min_lr for each iteration 
        self.warmup_epoch = 10  # warmup epcch
        self.verbose = verbose # log
        self.lrates = list() # for graph
        
        
    def cosine_annealing(self, epoch, epochs_per_cycle, max_lr):
        self.epochs += 1; 
        cos_inner = (math.pi * (self.epochs % epochs_per_cycle)) / (epochs_per_cycle)
        self.learning_rate = max_lr/2 * (math.cos(cos_inner) + 1)
        
        if ((self.epochs % epochs_per_cycle) == (epochs_per_cycle-1)):
            self.steps += 1
            self.max_lr *= 0.8
            self.epochs = 0;
            self.epochs_per_cycle = math.floor(self.epochs_per_cycle*1.2)
            
        return max_lr/2 * (math.cos(cos_inner) + 1)
  
    def warm_up(self, epoch):
        
        self.learning_rate = self.max_lr * epoch / self.warmup_epoch
        
        return self.learning_rate

    # calculate and set learning rate at the start of the epoch
    def on_epoch_begin(self, epoch, logs = None):
        if (epoch < self.warmup_epoch):
            # warm up learning rate
            lr = self.warm_up(epoch)
       
        elif(self.steps < self.iteration):
            # calculate cosine learning rate
            lr = self.cosine_annealing(epoch, self.epochs_per_cycle, self.max_lr)
            
        else:
            lr = self.min_lr
        
        if (self.verbose == 1):
            print('\nEpoch %05d: CosineAnnealingScheduler setting learng rate to %s.' % (epoch + 1, lr))  

        
        K.set_value(self.model.optimizer.lr, lr)

        self.lrates.append(lr)

# Train

In [30]:
def deep(imgs_train,imgs_mask_train,path,batch_size = 4,epochs = 10,image_size=512): 
    gpus = tf.config.experimental.list_logical_devices('GPU')
    print(gpus)
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
    
    with strategy.scope():
        model = hourglass_network(input_shape=(image_size, image_size, 1), num_classes=16, num_stacks=1, num_channels=32)
        model.compile(optimizer=Adam(lr=0.001), loss=dice_coef_loss, 
                        metrics=['accuracy', sens, dice_coef_loss])
    
    check_model_path = path+'A_check/'
    predict_path = path+'A_pred/'
    mkfolder(check_model_path)
    mkfolder(predict_path)

    model_checkpoint = ModelCheckpoint(check_model_path + 'final3_{epoch:d}_{loss:f}.hdf5', 
                                        monitor='val_dice_coef_loss',verbose=1, 
                                        save_best_only=False)
    earlystopping = EarlyStopping(monitor='val_dice_coef_loss', patience=30, restore_best_weights=True)
    cosine_schedule = CosineAnnealingWarmup(epochs_per_cycle=60, iteration=1,max_lr = 1e-3, min_lr = 1e-6)
    
    print('Fitting model...')
    model.fit(imgs_train, imgs_mask_train, batch_size=batch_size, epochs=epochs, verbose=1, 
              validation_split=0.2, shuffle=True, callbacks=[model_checkpoint, cosine_schedule ,earlystopping])
    print('save model')
    model.save(predict_path + 'final3.h5')
    return model


# Predict

In [31]:
def predict_save(pred_list,name_list):
    pred_img_path = '/home/hackerton/jupyter/Spine_data/A_pred/'
    if not os.path.isdir(pred_img_path):
        os.makedirs(pred_img_path)

    imgs = pred_list
    for i in range(imgs.shape[0]):
        img = imgs[i]
        img[img <= 0.5] = 0
        img[img > 0.5] = 255
        img = array_to_img(img)
        img.save(pred_img_path+"%s_pred.png" %(name_list[i]))

def predict_val(model,test_path,image_size=512):
    test_list = glob.glob(test_path+'*.dcm')
#     data_pre(test_list)
    
    img_mask_path ='/home/hackerton/jupyter/Spine_data/A_img_mask/'
    
    imgs_test, imgs_label_test, test_name = create_test_data(img_mask_path, image_size, image_size, 'test', 'jpg')
    print('predict test data')
    
    imgs_label_pred = model.predict(imgs_test, batch_size=4, verbose=1)
    name_list=test_name
    df = pd.DataFrame(columns=['name', 'acc', 'sen', 'spe', 'dsc'],dtype = float)
    df = df.astype({'name': 'str'})

    true_list=imgs_label_test
    print(true_list.shape)

    pred_list=imgs_label_pred
    print(np.unique(pred_list))
    pred_list[pred_list > 0.5] = 1
    pred_list[pred_list <= 0.5] = 0
    
    sensitivity=[]
    specificity=[]
    acc=[]
    dsc=[]

    for i in range(len(true_list)):
        yt=true_list[i].flatten()
        yp=pred_list[i].flatten()
        mat=confusion_matrix(yt,yp)
        if len(mat) == 2:
            ac=(mat[1,1]+mat[0,0])/(mat[1,0]+mat[1,1]+mat[0,1]+mat[0,0])
            st=mat[1,1]/(mat[1,0]+mat[1,1])
            sp=mat[0,0]/(mat[0,1]+mat[0,0])
            if mat[1,0]+mat[1,1] == 0:
                specificity.append(sp)
                acc.append(ac)
            else:
                sensitivity.append(st)  
                specificity.append(sp)
                acc.append(ac)
        else:
            specificity.append(1)
            acc.append(1)

        yt=true_list[i]
        yp=pred_list[i]
        if np.sum(yt) != 0 and np.sum(yp) != 0:
            dice = np.sum(yp[yt==1])*2.0 / (np.sum(yt) + np.sum(yp))
            dsc.append(dice)
        df=  df.append({'name':name_list[i], 'acc':ac, 'sen':st, 'spe':sp, 'dsc':dice}, ignore_index=True)

    print("complete")      
    print("acc avg : {0:0.4f}".format(np.mean(acc)))
    print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
    print("specificity avg : {0:0.4f}".format(np.mean(specificity)))
    print("dsc avg : {0:0.4f}".format(np.mean(dsc)))

    predict_save(pred_list,name_list)
    return test_name

In [32]:
def create_test_data(test_path, out_rows, out_cols, name, img_type):
    print('-'*30)
    print('Creating test images...')
    print('-'*30)

    i = 0
    imgs = glob.glob(test_path + "*." + img_type)
    imgdatas = np.ndarray((len(imgs),out_rows,out_cols,1), dtype=np.uint8)
    imglabels = np.ndarray((len(imgs),out_rows,out_cols,1), dtype=np.uint8)
    
    imgnames=[]
    for j, imgname in enumerate(imgs):
        if j%100==0:
            print('{}/{}'.format(j, len(imgs)))
        midname = imgname[imgname.rindex("/")+1:-4]
        img = load_img(imgname, color_mode = "grayscale")
        label = load_img(imgname.replace('jpg', 'png'), color_mode = "grayscale")
        img=img.resize((out_rows,out_cols))
        label=label.resize((out_rows,out_cols))

        img = img_to_array(img)
        label = img_to_array(label)
        imgdatas[j] = img
        imglabels[j] = label
        imgnames.append(midname)
         
    imgdatas = imgdatas.astype('uint8')
    imglabels = imglabels.astype('uint8')
    
    print('img : ', imgdatas.max())
    print('mask : ',imglabels.max())
    
    print('-'*30)
    print('normalization start...')
    print('-'*30)
    imgdatas = imgdatas/255.0
    
    imglabels[imglabels <= 127] = 0
    imglabels[imglabels > 127] = 1
    
    print('img : ',imgdatas.max())
    print('mask : ',imglabels.max())
    print('mask : ',imglabels.min())
    print('loading done')
    return(imgdatas,imglabels,imgnames)

# Evaluate

In [33]:
def get_results(test_name):
    print(len(test_name))
    angle_list = []
    dist_list = []
    
    # pred_img 개인서버 저장경로.
    pred_img_path = '/home/hackerton/jupyter/Spine_data/A_pred/'

    for i in range(len(test_name)):
        img = cv2.imread(pred_img_path+"%s_pred.png" %(test_name[i]))
        dst = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        contours = cv2.findContours(dst, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        
        count = 0
        for i in range(len(contours[0])):
            if contours[0][i].size < 25:
                continue
            if count == 2:
                break
            
            rect = cv2.minAreaRect(contours[0][i])
            box = cv2.boxPoints(rect)
            if (count == 0):
                if rect[2] < 45 :
                    x1 = box[1][0] # 아래 사각형 왼쪽 위
                    y1 = box[1][1]
                    x2 = box[2][0] # 아래 사각형 오른쪽 위
                    y2 = box[2][1]
                elif rect[2] >= 45 : 
                    x1 = box[0][0] # 아래 사각형 왼쪽 위
                    y1 = box[0][1]
                    x2 = box[1][0] # 아래 사각형 오른쪽 위
                    y2 = box[1][1]
                count += 1
            elif (count == 1):
                if rect[2] < 45 :
                    x3 = box[0][0] # 위 사각형 왼쪽 아래
                    y3 = box[0][1]
                    x4 = box[3][0] # 위 사각형 오른쪽 아래
                    y4 = box[3][1]
                elif rect[2] >= 45 :
                    x3 = box[3][0] # 위 사각형 왼쪽 아래
                    y3 = box[3][1]
                    x4 = box[2][0] # 위 사각형 오른쪽 아래
                    y4 = box[2][1] 
                count += 1
        
        l1 = (y1-y2)/(x1-x2)
        l2 = (y3-y4)/(x3-x4)
        if l1*l2 <= 0:
            pred_angle = np.arctan(np.abs(l1))*180/np.pi + np.arctan(np.abs(l2))*180/np.pi
        elif l1*l2 > 0:
            pred_angle = np.abs(np.arctan(np.abs(l1))*180/np.pi - np.arctan(np.abs(l2))*180/np.pi)                
        
        angle_list.append(pred_angle)       
        
        center_x1 = (x1+x2)/2
        center_y1 = (y1+y2)/2
        center_x2 = (x3+x4)/2
        center_y2 = (y3+y4)/2
        distance = np.sqrt((center_x1-center_x2)**2 + (center_y1-center_y2)**2)
        
        dist_list.append(distance) 
        
    return angle_list,dist_list

def get_score(angle_list,dist_list,test_name,test_path):
    angle_test = []
    angle_ai = []
    dist_test = []
    dist_ai = []
    get_no45 = []
    
    for i in range(len(test_name)):
        name = test_name[i]
        data = []
        
        jsonfile = test_path+'/{}.json'.format(name)
        for line in open(jsonfile,'r'):
            data.append(json.loads(line))
        for json_data in data:
            check = 0
            if json_data['annotation']['ANNOTATION_DATA'] is not None:
                for m in json_data['annotation']['ANNOTATION_DATA']:
                    if m['type']=='cobbAngle':
                        if m['label'] == 'L4-5A':
                            angle_test.append(m['angle'])
                            angle_ai.append(angle_list[i])
                            print(angle_list[i])
                            check = 1
                    elif m['type']=='line':
                        if m['label'] == 'L4-5H':
                            dist_test.append(m['distMm'])
                            dist_ai.append(dist_list[i])
                            check = 1
                if check==0:
                    get_no45.append(name)
    print(get_no45)
    print(r2_score(angle_ai, angle_test))
    print(r2_score(dist_ai, dist_test))

# Main

In [34]:
def main():
    # 주어진 Train Dataset 경로.
    train_path = '/mnt/hackerton/dataset/Dataset/Train/spine/'
    image_size = 512
    epochs = 100
    batch_sizes = 8
    # check, pred 저장할 개인서버 저장경로.
    path = '/home/hackerton/jupyter/Spine_data/'
    # 주어진 Train Dataset 경로.(Train Dataset의 일부를 Test Dataset으로 씀.)
    test_path = '/mnt/hackerton/dataset/Dataset/Train/spine/'
    
    imgs_train, imgs_mask_train, imgs_name = train_data_loading(train_path, image_size = image_size)
    model = deep(imgs_train, imgs_mask_train, path, batch_size = batch_sizes, epochs = epochs, image_size=image_size)

# 학습 안하고 가중치 파일을 이용해 평가시 위에 2줄 주석처리. 아래 코드 주석 해제.
#     from tensorflow.keras import Model
#     model = hourglass_network(input_shape=(512, 512, 1), num_classes=16, num_stacks=1, num_channels=32)
#     model.compile(optimizer=Adam(lr=0.001), loss=dice_coef_loss, metrics=['accuracy', sens, dice_coef_loss])
#     model.load_weights('/home/hackerton/jupyter/Spine_data/A_pred/j1.h5')
    
    test_name = predict_val(model, test_path, image_size = image_size)
    angle_list,dist_list = get_results(test_name)
    get_score(angle_list, dist_list, test_name, test_path)
    
if __name__ == "__main__":
    main()

------------------------------
Creating training images...
------------------------------
0/2390
1000/2390
2000/2390
img :  255.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
mask :  0.0
loading done
[LogicalDevice(name='/job:localhost/replica:0/task:0/device:GPU:0', device_type='GPU'), LogicalDevice(name='/job:localhost/replica:0/task:0/device:GPU:1', device_type='GPU')]
Tensor("conv2d_2/Identity:0", shape=(None, 512, 512, 1), dtype=float32)
Tensor("conv2d_2_target:0", shape=(None, None, None, None), dtype=float32)
Tensor("conv2d_2/Identity:0", shape=(None, 512, 512, 1), dtype=float32)
Tensor("conv2d_2_target:0", shape=(None, None, None, None), dtype=float32)
Fitting model...
Train on 1912 samples, validate on 478 samples

Epoch 00001: CosineAnnealingScheduler setting learng rate to 0.0.
Epoch 1/100
INFO:tensorflow:batch_all_reduce: 230 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_byt

1904/1912 [============================>.] - ETA: 0s - loss: 0.1702 - accuracy: 0.9596 - sens: 0.8485 - dice_coef_loss: 0.1702
Epoch 00011: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_11_0.170050.hdf5
1912/1912 [==============================] - 18s 9ms/sample - loss: 0.1700 - accuracy: 0.9597 - sens: 0.8488 - dice_coef_loss: 0.1700 - val_loss: 0.3924 - val_accuracy: 0.9286 - val_sens: 0.5347 - val_dice_coef_loss: 0.3949

Epoch 00012: CosineAnnealingScheduler setting learng rate to 0.0009972609476841367.
Epoch 12/100
1904/1912 [============================>.] - ETA: 0s - loss: 0.1617 - accuracy: 0.9616 - sens: 0.8539 - dice_coef_loss: 0.1617
Epoch 00012: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_12_0.161863.hdf5
1912/1912 [==============================] - 17s 9ms/sample - loss: 0.1619 - accuracy: 0.9616 - sens: 0.8538 - dice_coef_loss: 0.1619 - val_loss: 0.4410 - val_accuracy: 0.9256 - val_sens: 0.4700 - val_dice_coef_loss: 0.4416

Epoch 0

1904/1912 [============================>.] - ETA: 0s - loss: 0.0993 - accuracy: 0.9765 - sens: 0.9034 - dice_coef_loss: 0.0993
Epoch 00026: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_26_0.099425.hdf5
1912/1912 [==============================] - 18s 9ms/sample - loss: 0.0994 - accuracy: 0.9764 - sens: 0.9031 - dice_coef_loss: 0.0994 - val_loss: 0.9972 - val_accuracy: 0.8799 - val_sens: 0.0014 - val_dice_coef_loss: 0.9972

Epoch 00027: CosineAnnealingScheduler setting learng rate to 0.0008146601955249188.
Epoch 27/100
1904/1912 [============================>.] - ETA: 0s - loss: 0.0990 - accuracy: 0.9765 - sens: 0.9043 - dice_coef_loss: 0.0990
Epoch 00027: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_27_0.098995.hdf5
1912/1912 [==============================] - 17s 9ms/sample - loss: 0.0990 - accuracy: 0.9765 - sens: 0.9042 - dice_coef_loss: 0.0990 - val_loss: 0.5992 - val_accuracy: 0.9051 - val_sens: 0.2996 - val_dice_coef_loss: 0.6007

Epoch 0

1904/1912 [============================>.] - ETA: 0s - loss: 0.0771 - accuracy: 0.9816 - sens: 0.9255 - dice_coef_loss: 0.0771
Epoch 00041: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_41_0.077187.hdf5
1912/1912 [==============================] - 18s 9ms/sample - loss: 0.0772 - accuracy: 0.9815 - sens: 0.9255 - dice_coef_loss: 0.0772 - val_loss: 0.1792 - val_accuracy: 0.9601 - val_sens: 0.7953 - val_dice_coef_loss: 0.1796

Epoch 00042: CosineAnnealingScheduler setting learng rate to 0.00044773576836617336.
Epoch 42/100
1904/1912 [============================>.] - ETA: 0s - loss: 0.0745 - accuracy: 0.9822 - sens: 0.9284 - dice_coef_loss: 0.0745
Epoch 00042: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_42_0.074499.hdf5
1912/1912 [==============================] - 18s 9ms/sample - loss: 0.0745 - accuracy: 0.9822 - sens: 0.9283 - dice_coef_loss: 0.0745 - val_loss: 0.3524 - val_accuracy: 0.9397 - val_sens: 0.5448 - val_dice_coef_loss: 0.3533

Epoch 

1904/1912 [============================>.] - ETA: 0s - loss: 0.0647 - accuracy: 0.9846 - sens: 0.9371 - dice_coef_loss: 0.0647
Epoch 00056: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_56_0.064777.hdf5
1912/1912 [==============================] - 18s 9ms/sample - loss: 0.0648 - accuracy: 0.9845 - sens: 0.9368 - dice_coef_loss: 0.0648 - val_loss: 0.1314 - val_accuracy: 0.9697 - val_sens: 0.8694 - val_dice_coef_loss: 0.1312

Epoch 00057: CosineAnnealingScheduler setting learng rate to 0.00011142701927151455.
Epoch 57/100
1904/1912 [============================>.] - ETA: 0s - loss: 0.0639 - accuracy: 0.9847 - sens: 0.9382 - dice_coef_loss: 0.0639
Epoch 00057: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_57_0.063825.hdf5
1912/1912 [==============================] - 18s 9ms/sample - loss: 0.0638 - accuracy: 0.9847 - sens: 0.9382 - dice_coef_loss: 0.0638 - val_loss: 0.1346 - val_accuracy: 0.9694 - val_sens: 0.8485 - val_dice_coef_loss: 0.1346

Epoch 

1904/1912 [============================>.] - ETA: 0s - loss: 0.0627 - accuracy: 0.9851 - sens: 0.9397 - dice_coef_loss: 0.0627
Epoch 00071: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_71_0.062798.hdf5
1912/1912 [==============================] - 18s 9ms/sample - loss: 0.0628 - accuracy: 0.9850 - sens: 0.9395 - dice_coef_loss: 0.0628 - val_loss: 0.1179 - val_accuracy: 0.9720 - val_sens: 0.8864 - val_dice_coef_loss: 0.1179

Epoch 00072: CosineAnnealingScheduler setting learng rate to 1e-06.
Epoch 72/100
1904/1912 [============================>.] - ETA: 0s - loss: 0.0620 - accuracy: 0.9852 - sens: 0.9401 - dice_coef_loss: 0.0620
Epoch 00072: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_72_0.062057.hdf5
1912/1912 [==============================] - 18s 9ms/sample - loss: 0.0621 - accuracy: 0.9852 - sens: 0.9402 - dice_coef_loss: 0.0621 - val_loss: 0.1172 - val_accuracy: 0.9722 - val_sens: 0.8872 - val_dice_coef_loss: 0.1172

Epoch 00073: CosineAnne


Epoch 00087: CosineAnnealingScheduler setting learng rate to 1e-06.
Epoch 87/100
1904/1912 [============================>.] - ETA: 0s - loss: 0.0621 - accuracy: 0.9851 - sens: 0.9400 - dice_coef_loss: 0.0621
Epoch 00087: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_87_0.062165.hdf5
1912/1912 [==============================] - 18s 9ms/sample - loss: 0.0622 - accuracy: 0.9851 - sens: 0.9400 - dice_coef_loss: 0.0622 - val_loss: 0.1174 - val_accuracy: 0.9721 - val_sens: 0.8875 - val_dice_coef_loss: 0.1174

Epoch 00088: CosineAnnealingScheduler setting learng rate to 1e-06.
Epoch 88/100
1904/1912 [============================>.] - ETA: 0s - loss: 0.0625 - accuracy: 0.9851 - sens: 0.9394 - dice_coef_loss: 0.0625
Epoch 00088: saving model to /home/hackerton/jupyter/Spine_data/A_check/final3_88_0.062570.hdf5
1912/1912 [==============================] - 18s 9ms/sample - loss: 0.0626 - accuracy: 0.9851 - sens: 0.9394 - dice_coef_loss: 0.0626 - val_loss: 0.1178 - val_accurac

(239, 512, 512, 1)
[0.0000000e+00 2.9802322e-08 5.9604645e-08 ... 9.9999988e-01 9.9999994e-01
 1.0000000e+00]
complete
acc avg : 0.9911
sensitivity avg : 0.8603
specificity avg : 0.9957
dsc avg : 0.8651
239


/home/hackerton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in float_scalars


30.121705414325575
3.576328243997108
1.5785728304058377
4.289126572336227
3.583330682000252
25.20114065982636
8.014034055782739
20.39282238037349
30.411054852568938
36.86987340391658
22.011295681806665
11.104469045527487
12.80424230209473
11.449354325025404
9.462305873288361
2.4194839497898277
14.869960445805155
0.0
8.698153913900855
6.6048142717434875
4.969789074073848
23.36068941201895
6.241274464961795
7.594630772109748
8.343909354249316
5.8308219307526965
6.095866567721394
9.491105579367131
12.753276711168564
16.025589179783584
15.914374810353852
12.528808093624797
11.003549158559139
5.303703789104255
18.935289466683567
7.823106576262198
6.911214850362967
16.440402789777735
12.634528322919193
21.801420830641685
8.574257484007997
12.447254354274374
18.86083784340949
0.41518286709615726
12.777839992414634
4.172527559241293
18.434964022714972
0.6780170840947104
15.851801206211546
12.094781588382219
7.281511635224503
6.115517448850344
27.70174036308608
7.125027092141045
4.5171132709110